In [2]:
# 导入必要的库
import os
import paddle
import json
from paddlenlp import Taskflow
from collections import defaultdict
# 查看PaddleNLP版本
# print(paddlenlp.__version__)

D:\env\anaconda3\envs\pytorch\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:

# 使用训练后的模型进行预测
schema = [{'评价维度': ['观点词', '情感倾向[积极,消极,中立]']}]
senta = Taskflow("sentiment_analysis", model='uie-senta-base', 
                 schema=schema, task_path='.\\checkpoint\\model_best')
senta("这部电影的特效非常好")

[2023-09-04 23:56:49,738] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load '.\checkpoint\model_best'.
D:\env\anaconda3\envs\pytorch\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:2481: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  FutureWarning,
D:\env\anaconda3\envs\pytorch\lib\site-packages\paddlenlp\transformers\tokenizer_utils_base.py:1884: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[{'评价维度': [{'text': '特效',
    'start': 5,
    'end': 7,
    'probability': 0.9981206265744618,
    'relations': {'观点词': [{'text': '非常好',
       'start': 7,
       'end': 10,
       'probability': 0.9037809932659293}],
     '情感倾向[积极,消极,中立]': [{'text': '积极', 'probability': 0.9998376426356259}]}}]}]

### 对所有的评论进行属性和观点提取

In [35]:
import pandas as pd
comments = pd.read_csv('./data/评论数据.txt', encoding='utf-8', names=['评论'], quoting=3)
comments

,评论
0,的声音
1,“那是不可能的”我的意思是，他不可能复制自己，而且已经复制了很多次了
2,请制作第十一部分。
3,偏头痛的魔咒终于回来了
4,2:40我喜欢这个
...,...
5638,"""It's a drag"
5639,"""While its nearly three hour running time is a..."
5640,"""It is this combination of maximalism"
5641,这是多年来任何一部美国大片，我相信大多数负面评论只是中国版“洗白”的一个分支。英雄和决策者都...


In [54]:
# 提取属性词和观点词，并进行拼接
def extract_prop_and_op(text, predict):
    results = predict(text)[0]
    if results.get('评价维度') is None:
        return []
    results = results['评价维度']
    prop_op_lst = []
    for res in results:
        prop = res['text']
        if res.get('relations') is None or res.get('relations').get('观点词') is None:
            continue
        relations = res['relations']['观点词']
        for (idx, relation) in enumerate(relations):
            if idx == 0:
                prop += ': '
            else:
                prop += ';'
            prop += relation['text']
            
        prop_op_lst.append(prop)
    return prop_op_lst

In [55]:
# 使用训练的模型提取评论的属性词和观点词
# extract_prop_and_op("房间很大，很舒服", senta)
subject_lst = []
for comment in comments['评论']:
    subject_lst.extend(extract_prop_and_op(comment, senta))
res_dat = pd.DataFrame(data=subject_lst, columns=['opinion'])
print(res_dat)


                            opinion
0                        偏头痛的魔咒: 回来
1                            这个: 喜欢
2                           集: 间隔多久
3                        《西游记》: 懒得读
4                         视频质量: 刚刚好
...                             ...
3778  It: combination of maximalism
3779                         电影: 优秀
3780                      这部电影: 很有趣
3781                《流感》和《怪物》: 排名第一
3782                     特效: 简直令人惊叹

[3783 rows x 1 columns]


In [56]:
# 将结果保存到文件
res_dat.to_csv('./data/评价观点抽取.csv', encoding='utf-8', index=None)

In [51]:
x = [1,2,4]
x.extend([])
x

[1, 2, 4]

In [69]:
# 去除重复值
import csv
res_dat2 = pd.DataFrame(data=res_dat['opinion'].drop_duplicates().apply(lambda x: x.replace(',', ';')), columns=['opinion'])
res_dat2.to_csv('./data/评价观点抽取.csv', encoding='utf-8', index=None)

In [70]:
res_dat2.head(20)
print(res_dat2.info)

<bound method DataFrame.info of                             opinion
0                        偏头痛的魔咒: 回来
1                            这个: 喜欢
2                           集: 间隔多久
3                        《西游记》: 懒得读
4                         视频质量: 刚刚好
...                             ...
3777    action: Entertaining enough
3778  It: combination of maximalism
3780                      这部电影: 很有趣
3781                《流感》和《怪物》: 排名第一
3782                     特效: 简直令人惊叹

[3426 rows x 1 columns]>
